# The Energy of a Quantum Physical Two-Body System

In [47]:
import numpy as np
import matplotlib.pyplot as plt

## Constants

In [48]:
sigma = 1
x0 = 0

planck = 6.626e-34
reduced_planck = planck/(2*np.pi)
e_mass = 9.109e-31

## Discretization

In [49]:
# Length of interval
L = 10 
# No. of subintervals
N = 11*100
# Stepsize
h = L / (N-1)

# Argument vector
xi = np.linspace(-L/2, L/2, N)

## Functions

In [50]:
def v_func(x):
    return x**2/2

In [122]:
def psi_func(x, *args):
    x0, sigma = args
    return (1/np.sqrt(2*sigma))*np.exp(-((x-x0)**2)/2*sigma**2)

In [107]:
def populate_vector(func, *args):
    vector = [func(xi[i], *args) for i in range(N)]
    return np.array(vector).reshape(-1, 1)

In [124]:
def finite_difference_scheme():
    m = np.zeros((N,N))
    for i in range(N):
        m[i,i] = -2
        if i+1 < N:
            m[i,i+1] = 1
        if i-1 >= 0:
            m[i, i-1] = 1
    return m

In [147]:
def compute_e(x0, sigma):
    psi_vector = populate_vector(psi_func, x0, sigma)
    v_vector = populate_vector(v_func)
    finite_difference_matrix = finite_difference_scheme()
    #kinetic_constant = -reduced_planck**2/(2*e_mass) # Kan være 1 inntil videre
    h_matrix = (1/h**2)*(psi_vector * finite_difference_matrix) + (v_vector * psi_vector)
    
    return h*(psi_vector.T @ h_matrix) @ psi_vector

## Computing the Energy

In [145]:
e = compute_e(x0=0, sigma=1)

In [146]:
e

array([[42.81650072]])